## Initialization

In [1]:
import numpy as np

In [2]:
magnObj = 60
nrCamPix = 16 # num pixels behind lenslet
camPixPitch = 6.5
microLensPitch = nrCamPix*camPixPitch/magnObj
voxPitch = microLensPitch/5

The number of voxels along each side length of the cube is determined by the voxPitch. An odd number of voxels will allow the center of a voxel in the center of object space.
Object space center:
voxCtr:center voxel where all rays of the central microlense converge
volCtr:same center in micrometers

In [3]:
voxNrX = round(250/voxPitch)
if voxNrX % 2 == 1:
    voxNrX += 1
voxNrYZ = round(700/voxPitch)
if voxNrYZ % 2 == 1:
    voxNrYZ += 1
voxCtr = np.array([voxNrX/2, voxNrYZ/2, voxNrYZ/2])
volCtr = voxCtr*voxPitch
volCtr

array([125.14666667, 350.13333333, 350.13333333])

In [4]:
wavelength = 0.550
naObj = 1.2
nMedium = 1.52


What are they different unit types?
What is the central lenslet?

Finding angles to/between central lenset. Basically, the angle going to each of the 16 pixels for each microlens.

In [5]:
microLensCtr = [8, 8] # (unit: camera pixels)
rNA = 7.5 # radius of edge of objective lens (unit:camera pixels)
camPixRays = np.zeros([nrCamPix, nrCamPix])
i = np.linspace(1, nrCamPix, nrCamPix)
j = np.linspace(1, nrCamPix, nrCamPix)
jv, iv = np.meshgrid(i, j) # row/column defined instead of by coordinate
distFromCtr = np.sqrt((iv-0.5-microLensCtr[0])**2 + (jv-0.5-microLensCtr[1])**2)
camPixRays[distFromCtr > rNA] = np.NaN
iRel2Ctr = iv-0.5-microLensCtr[0]
jRel2Ctr = jv-0.5-microLensCtr[1]
camPixRaysAzim = np.round(np.rad2deg(np.arctan2(jRel2Ctr, iRel2Ctr)))
camPixRaysAzim[distFromCtr > rNA] = np.NaN
distFromCtr[distFromCtr > rNA] = np.NaN
camPixRaysTilt = np.round(np.rad2deg(np.arcsin(distFromCtr/rNA*naObj/nMedium)))

# posRel2Ctr[distFromCtr > rNA] = np.NaN
# camPixRaysAzim = np.arctan2(jv-0.5-microLensCtr[1], iv-0.5-microLensCtr[0])/(np.pi/180)
# for i in range(nrCamPix):
#     for j in range(nrCamPix):
#         if distFromCtr[i,j] > rNA:
#             camPixRays[i,j] = np.NaN
#         else:
#             pass

Camera ray entrance. For each inital ray position, we find the position on the entrance face of the object cube for which the ray enters.
This is bascially the same as "rayEnter". Here $x=0$.

In [6]:
camRayEntranceX = np.zeros([nrCamPix, nrCamPix])
camRayEntranceY = volCtr[0]*np.tan(np.deg2rad(camPixRaysTilt))*np.sin(np.deg2rad(camPixRaysAzim))+volCtr[1]
camRayEntranceZ = volCtr[0]*np.tan(np.deg2rad(camPixRaysTilt))*np.cos(np.deg2rad(camPixRaysAzim))+volCtr[2]
camRayEntranceX[np.isnan(camRayEntranceY)] = np.NaN
camRayEntranceY[1,6]
# When accessing array, x & y orientation are the same, 
# but minus one needs to be applied to match up with the mathematica indices.

nrRays = np.sum(~np.isnan(camRayEntranceY)) # Number of all rays in use

In [12]:
camRayEntrance = np.array([camRayEntranceX, camRayEntranceY, camRayEntranceZ])
camRayEntrance[:,1,6]
rayEnter = camRayEntrance.copy()
volCtrGridTemp = np.array([np.full((nrCamPix,nrCamPix), volCtr[i]) for i in range(3)])
rayExit = rayEnter + 2 * (volCtrGridTemp - rayEnter)
rayEnter[:,1,1]

array([nan, nan, nan])

Testing the rayExit generate to only one element. This is the first element shown in the Mathematica code.

In [8]:
# rayExit = np.zeros((3,16,16))
print(rayEnter[:,1,4])
# rayExit[:,1,4] = rayEnter[:,1,4] + 2*(volCtr-rayEnter[:,1,4])
print(rayExit[:,1,4])

[  0.         277.57966622 213.67973144]
[250.29333333 422.68700045 486.58693523]


In [9]:
rayDiff = rayExit - rayEnter
for i in range(nrCamPix):
    for j in range(nrCamPix):
        rayDiff[:,i,j] = rayDiff[:,i,j] / np.linalg.norm(rayDiff[:,i,j])
print(rayDiff[:,1,4])
# Remaining of rayunitvectors needed for simulation

[0.62932039 0.36484793 0.68617916]


In [10]:
# print(yv)
# print(camPix)
# print(distFromCtr)
print(camPixRaysTilt)
print(camPixRaysAzim)
print(type(camPixRaysAzim[0,0]))
print(camPixRays)

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan 51. 47. 45. 43. 43. 45. 47. 51. nan nan nan nan]
 [nan nan nan 48. 43. 39. 37. 36. 36. 37. 39. 43. 48. nan nan nan]
 [nan nan 48. 42. 37. 33. 30. 28. 28. 30. 33. 37. 42. 48. nan nan]
 [nan 51. 43. 37. 31. 27. 24. 22. 22. 24. 27. 31. 37. 43. 51. nan]
 [nan 47. 39. 33. 27. 22. 18. 16. 16. 18. 22. 27. 33. 39. 47. nan]
 [nan 45. 37. 30. 24. 18. 13. 10. 10. 13. 18. 24. 30. 37. 45. nan]
 [nan 43. 36. 28. 22. 16. 10.  4.  4. 10. 16. 22. 28. 36. 43. nan]
 [nan 43. 36. 28. 22. 16. 10.  4.  4. 10. 16. 22. 28. 36. 43. nan]
 [nan 45. 37. 30. 24. 18. 13. 10. 10. 13. 18. 24. 30. 37. 45. nan]
 [nan 47. 39. 33. 27. 22. 18. 16. 16. 18. 22. 27. 33. 39. 47. nan]
 [nan 51. 43. 37. 31. 27. 24. 22. 22. 24. 27. 31. 37. 43. 51. nan]
 [nan nan 48. 42. 37. 33. 30. 28. 28. 30. 33. 37. 42. 48. nan nan]
 [nan nan nan 48. 43. 39. 37. 36. 36. 37. 39. 43. 48. nan nan nan]
 [nan nan nan nan 51. 47. 45. 43. 43. 45. 47. 51. nan nan nan 